In [1]:
import sys
import numpy as np
import copy
import os
import torch
import torchvision.transforms as TF
import copy
sys.path.append('../src/')

import tforms 
import utils

import inaturalist_dataset as dset

import novelty_dfm_CL.novelty_dataset_wrappers as novdset

# Get Inaturalist Dataset

In [2]:

transform = TF.Compose([TF.Resize(256), TF.CenterCrop(224), TF.RandomHorizontalFlip(), TF.ToTensor()])
dataroot = '../data/inaturalist/'

dataset = dset.WrapNaturalist(dataroot, version='2019', target_type=['super'], \
                                transform=transform, target_transform=None, download=False)

task_indices_new = np.random.randint(0, dataset.__len__(), size=(5000,))
task_indices_old1 = np.random.randint(0, dataset.__len__(), size=(5000,))
task_indices_old2 = np.random.randint(0, dataset.__len__(), size=(5000,))

np.save('../sandbox/filetask_new.npy', task_indices_new)
np.save('../sandbox/filetask_old1.npy', task_indices_old1)
np.save('../sandbox/filetask_old2.npy', task_indices_old2)

dataset_task_new = dset.inaturalistTask(dataroot, tasklist='../sandbox/filetask_new.npy', transform=transform)

dataset_task_old1 = dset.inaturalistTask(dataroot, tasklist='../sandbox/filetask_old1.npy', transform=transform)
dataset_task_old2 = dset.inaturalistTask(dataroot, tasklist='../sandbox/filetask_old2.npy', transform=transform)

datasets_old = [dataset_task_old1, dataset_task_old2]


In [8]:

class NovelTask():
    def __init__(self, task, num_current_classes, current_dset, \
        pred_novel_inds=None, returnGT=False):
        """ 
        When Each novel task only contains one class/label (except for first task)
        Filter and keep only predicted_novel_inds
        Return Ground Truths Optionally
        """

        self.dset = current_dset
        self.returnGT = returnGT
        self.task = task
        self.num_current_classes = num_current_classes
        
        if pred_novel_inds is not None:
            self.dset.select_specific_subset(pred_novel_inds)
        self.indices_task = self.dset.indices_task


    def __len__(self):
        return len(self.indices_task)

        
    def __getitem__(self, idx):
        
        im, class_gt, novelty_gt = self.dset.__getitem__(idx)

        if self.task>0:
            class_lbl = self.num_current_classes
        else:
            class_lbl = class_gt

        if self.returnGT:
            return im, class_lbl, class_gt, novelty_gt
            
        return im, class_lbl




current_dset = novdset.CurrentTask(dataset_task_new, datasets_old, returnIDX=False)

current_dset.__len__(), current_dset.indices_task[0]


indices_select = np.random.randint(0,15000, size=(1000,))


p_dset = NovelTask(1, 2, current_dset, pred_novel_inds=indices_select, returnGT=True)

p_dset.__len__(), p_dset.__getitem__(10)


dset_id 0


(1000,
 (tensor([[[0.3647, 0.3686, 0.3804,  ..., 0.4824, 0.4549, 0.4000],
           [0.3529, 0.3608, 0.3765,  ..., 0.4902, 0.4078, 0.3569],
           [0.3451, 0.3490, 0.3686,  ..., 0.4039, 0.3451, 0.3294],
           ...,
           [0.1176, 0.1098, 0.1098,  ..., 0.1608, 0.1529, 0.1490],
           [0.1098, 0.1098, 0.1216,  ..., 0.1843, 0.1647, 0.1490],
           [0.1098, 0.1137, 0.1137,  ..., 0.2667, 0.1882, 0.1569]],
  
          [[0.4824, 0.4824, 0.4824,  ..., 0.5647, 0.5098, 0.4510],
           [0.4784, 0.4745, 0.4784,  ..., 0.5451, 0.4549, 0.4078],
           [0.4745, 0.4706, 0.4706,  ..., 0.4588, 0.4000, 0.3961],
           ...,
           [0.2078, 0.2000, 0.1961,  ..., 0.2706, 0.2667, 0.2588],
           [0.2039, 0.2039, 0.2078,  ..., 0.2745, 0.2745, 0.2706],
           [0.2118, 0.2157, 0.2118,  ..., 0.3137, 0.2824, 0.2784]],
  
          [[0.3804, 0.3804, 0.3804,  ..., 0.3451, 0.2863, 0.2275],
           [0.3725, 0.3725, 0.3765,  ..., 0.2627, 0.2039, 0.1608],
           [0.3

In [11]:

class NovelTask():
    def __init__(self, task, num_current_classes, current_dset, \
        pred_novel_inds=None, returnGT=False):
        """ 
        When Each novel task only contains one class/label (except for first task)
        Filter and keep only predicted_novel_inds
        Return Ground Truths Optionally
        """

        self.dset = current_dset
        self.returnGT = returnGT
        self.task = task
        self.num_current_classes = num_current_classes
        
        if pred_novel_inds is not None:
            self.dset.select_specific_subset(pred_novel_inds)
        self.indices_task = self.dset.indices_task


    def __len__(self):
        return len(self.indices_task)

        
    def __getitem__(self, idx):
        
        im, class_gt, novelty_gt = self.dset.__getitem__(idx)

        if self.task>0:
            class_lbl = self.num_current_classes
        else:
            class_lbl = class_gt

        if self.returnGT:
            novelty_gt = self.gt_novelty_y[idx]
            class_gt = self.gt_y[idx]
            return im, class_lbl, class_gt, novelty_gt
            
        return im, class_lbl



